Finetuning eines GPT-3-Modell

In diesem Notebook wird dein Datensatz zusammengestellt, der ausschließlich zum Finetuning eines GPT-3-Modells dient

In [2]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import re
import json
import openai
import langdetect

# Datensatz erstellen für GPT3 Finetuning 

In [ ]:
# df_rest = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Restaurants_2023-03-26_09-43-08-320.csv')
# df_act = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Activities_2023-03-26_11-08-15-435.csv')
# df_hotel = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Hotels_2023-03-26_11-34-16-492.csv')


# MIT VIEL MEHR DATEN UND DANN DAFÜR DIE LOCATIONS RAUSSCHMEIßEN

df_rest = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Restaurants_v2_2023-03-30_14-05-51-577.csv')
df_act = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Activities_v2_2023-04-05_08-52-36-619.csv')
df_hotel = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Hotels_v2_2023-04-05_09-21-00-200.csv')

print('Restaurants (Shape): ', df_rest.shape)
print('Activities (Shape): ', df_act.shape)
print('Hotels (Shape): ', df_hotel.shape)

In [ ]:
# add a column genre restaurant to the df_rest
df_rest['genre'] = 'restaurant'
df_act['genre'] = 'activity'
df_hotel['genre'] = 'hotel'

Idee: nur den Text berücksichtigen. Noch kein Fokus auf zwei Modalitäten

In [ ]:
df_complete_ft = pd.concat([df_rest, df_act, df_hotel], axis=0)
print('All (raw): ', df_complete_ft.shape)
df_complete_ft = df_complete_ft.reset_index(drop=True)
df_complete_ft = df_complete_ft.drop_duplicates(subset=['reviewId'])
print('All (raw): ', df_complete_ft.shape)
df_complete_ft = df_complete_ft.dropna(subset=['text'])
print('All (raw): ', df_complete_ft.shape)
df_complete_ft['text'] = df_complete_ft['text'].astype(str)
print('All with text: ', df_complete_ft.shape)
# save 
df_complete_ft.to_csv('01_Raw_Data\dataset_für_weitere_forschung.csv', index=False)

In [ ]:
df_complete_ft = pd.read_csv('01_Raw_Data\dataset_für_weitere_forschung.csv')
df_complete_ft = df_complete_ft[['text', 'reviewId', 'url', 'placeId','categoryName', 'stars','title', 'reviewImageUrls/0','genre']]

print('All with text: ', df_complete_ft.shape)
print('Unique categoryNames: ', df_complete_ft['categoryName'].nunique())
print('Unique categoryNames: ', df_complete_ft['categoryName'].unique())
print('Unique places: ', df_complete_ft['placeId'].nunique())

df_complete_ft.to_csv('01_Raw_Data\dataset_für_weitere_forschung_relCols.csv', index=False)

In [71]:
df_complete_ft = pd.read_csv('01_Raw_Data\dataset_für_weitere_forschung_relCols.csv')
df_reduced = pd.read_csv('real_base_sent_reduced.csv')

print('All places and all reviews: ', df_complete_ft.shape)

df_complete_ft = df_complete_ft[~df_complete_ft['placeId'].isin(df_reduced['placeId'])]
print('Not places from base dataset any more: ', df_complete_ft.shape)

print (df_complete_ft.groupby('categoryName')['stars'].mean())

df_complete_ft = df_complete_ft.dropna(subset=['reviewImageUrls/0'])
print('with images', df_complete_ft.shape)

print('All places and all reviews: ', df_complete_ft.shape)
print (df_complete_ft.groupby('categoryName')['stars'].mean())

df_complete_ft.to_csv('dataset_ft_v3.csv', index=False)

All places and all reviews:  (84831, 9)
Not places from base dataset any more:  (47431, 9)
categoryName
American restaurant            4.534658
Amusement Centre               4.571429
Amusement center               3.934783
Amusement park                 4.200000
Aquarium                       4.183500
Asian fusion restaurant        4.656766
Bar                            4.885057
Bar & grill                    4.356684
Bistro                         4.679688
Boat rental service            4.812500
Bowling alley                  4.463415
Buffet restaurant              3.837177
Casino                         3.594966
Children's Amusement Centre    4.213953
Chinese restaurant             3.751926
Escape room center             4.714286
Exhibit                        4.029412
Fine dining restaurant         4.569231
French restaurant              4.782383
German restaurant              4.522388
Go-kart track                  4.538793
Golf club                      4.468750
Golf course     

In [85]:
df = pd.read_csv('dataset_ft_v3_en.csv')
#filter out any non english reviews
def is_english(text):
    try:
        return langdetect.detect(text) == 'en'
    except:
        return False

print('Starting...', df.shape)
df = df[df['text'].apply(is_english)]

print('Finsihed only English', df.shape)
df.to_csv('dataset_ft_v3_en.csv', index=False)

Starting... (5759, 9)
Finsihed only English (5759, 9)


### Ab hier kann für weitere Finetuning Datasets angesetzt werde. ft_v3 beinhaltet keine Places aus dem Base mehr!!!!

In [11]:
df = pd.read_csv('dataset_ft_v3_en.csv')
print('', df.shape)

df = df.groupby('placeId').apply(lambda x: x.sample(min(len(x), 100)))

print(df['placeId'].nunique())
print(df['genre'].value_counts())
print (df.groupby('genre')['stars'].mean())
print('', df.shape)

df.to_csv('dataset_ft_v3_en_mulitmod.csv', index=False)

 (5759, 9)
107
hotel         1291
activity      1273
restaurant     633
Name: genre, dtype: int64
genre
activity      4.626080
hotel         4.340821
restaurant    4.552923
Name: stars, dtype: float64
 (3197, 9)


In [16]:
df = pd.read_csv("dataset_ft_v3_en_mulitmod.csv")
print(df.shape)
df = df[~df['text'].str.contains("END")]
print(df.shape)
df.to_csv('dataset_ft_v3_en_mulitmod.csv', index=False)

(3195, 9)
(3195, 9)


In [18]:
df_reduced = pd.read_csv("real_base_sent_reduced.csv")
df_fine_tuning = pd.read_csv("dataset_ft_v3_en_mulitmod.csv")
print(f"Rows that appear in both data sets:" , df_reduced['reviewId'].isin(df_fine_tuning['reviewId']).sum())
print(f"Places that appear in both data sets:" , df_reduced['placeId'].isin(df_fine_tuning['placeId']).sum())

Rows that appear in both data sets: 0
Places that appear in both data sets: 0


In [19]:
def generate_prompt_gpt3_v4(row):
    category = row["categoryName"]
    genre = row["genre"]
    if (genre == "activity"):
        genre = "activity or attraction"
    elif (genre == "hotel"):
        genre = "hotel"
    elif (genre == "restaurant"):
        genre = "restaurant"
    return f"A Google Maps review about a {genre}."

def generate_prompt_gpt3_v3(row):
    category = row["categoryName"].lower()
    return f"A Google Maps review about a {category}."

def generate_prompt_gpt3_v2(row):
    category = row["categoryName"].lower()
    return f"A Google Maps review about a {category}."

def generate_prompt_gpt3_v1(row):
    stars = row["stars"]
    category = row["categoryName"].lower()
    keywords = row["keywords_only"]
    return f"Write a {stars} stars Google Maps review about a {category}, with the following keywords: {keywords}."

# Ansatz verworfen.
def generate_prompt_gpt35turbo(row):
    category = row["categoryName"].lower()
    return f"Write a Google Maps review about a {category}."

# Ansatz verworfen.
def generate_prompt_gpt35turbo(row):
    stars = row["stars"]
    category = row["categoryName"].lower()
    keywords = row["keywords_only"]
    keywords = keywords[1:-1].split(",")[:3]
    keywords = [k.strip() for k in keywords]
    keywords = ", ".join(keywords)
    keywords = keywords.replace("'", "")
    return f"Write a {stars} stars Google Maps review about a {category}, with the following keywords: {keywords}."

In [20]:
df = pd.read_csv ("dataset_ft_v3_en_mulitmod.csv")

print(df.shape)
df["prompt"] = df.apply(generate_prompt_gpt3_v3, axis=1)
df["completion"] = df["text"]
print(df.columns)
print(df.shape)

df = df.drop(columns=['text', 'url', 'placeId', 'categoryName', 'stars', 'title', 'reviewImageUrls/0', 'genre'])
print(df.shape)
df.to_csv('dataset_ft_v3_en_mulitmod_pc.csv', index=False)

(3195, 9)
Index(['text', 'reviewId', 'url', 'placeId', 'categoryName', 'stars', 'title',
       'reviewImageUrls/0', 'genre', 'prompt', 'completion'],
      dtype='object')
(3195, 11)
(3195, 3)


In [208]:
# print a example row
df = pd.read_csv ("dataset_ft_v3_en_mulitmod_pc.csv")
# how many placeId are in the dataset
print(df['reviewId'].nunique())
df.shape

5756


(5756, 11)

# Archiv: für  FT Datensatz v2

In [ ]:
#  drop all rows with na in reviewImageUrls/0 in df_complete_ft
# df_reduced = pd.read_csv('real_base_sent_reduced.csv')

print('All with text: ', df_complete_ft.shape)
#df_complete_ft = df_complete_ft.dropna(subset=['reviewImageUrls/0'])
print('All with text: ', df_complete_ft.shape)

print('Unique categoryNames: ', df_complete_ft['categoryName'].nunique())
print('Unique categoryNames: ', df_complete_ft['categoryName'].unique())
print('Unique places: ', df_complete_ft['placeId'].nunique())

# per placeID keep at most 15 reviews
#df_complete_ft = df_complete_ft.groupby('placeId').head(15)


In [ ]:
# df_reduced = pd.read_csv('real_base_sent_reduced.csv')

# # drop all rows with na in reviewImageUrls/0
# print('All with text: ', df_reduced.shape)
# df_reduced = df_reduced.dropna(subset=['reviewImageUrls/0'])
# print('All with text: ', df_reduced.shape)


Jetzt. Andere Sprachen rausfiltern und Einträge die im Base-Datensatz sind rausfiltern. @TODO: RAUSFINDEN, OB AUF ANDERE LOCATOINS TRAINIEREN ODER DIE SELBEN WIE IM BASE_..._.csv Datensatz

In [ ]:
df = pd.read_csv ("finetuning_gpt3_v2.csv")
df.shape

In [ ]:
def is_english(text):
    try:
        return langdetect.detect(text) == 'en'
    except:
        return False

print('Starting...', df.shape)
df = df[df['text'].apply(is_english)]

print('Finsihed only English', df.shape)
df.to_csv('finetuning_gpt3_sameplaces_only_english.csv', index=False)

In [88]:
df = pd.read_csv ("finetuning_gpt3_sameplaces_only_english_multimod.csv")
print(df.shape)

df = df.groupby('placeId').head(25)
print(df['placeId'].nunique())

print(df.shape)
df.to_csv('finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv', index=False)

(5286, 7)
50
(893, 7)


In [ ]:
# Rows die END enthlaten rausnehmen, für COMPLETION PROMPT WICHTIG
df = pd.read_csv ("finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv")
print(df.shape)
df = df[~df['text'].str.contains("END")]
print(df.shape)
df.to_csv('finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv', index=False)

In [ ]:
df_reduced = pd.read_csv("real_base_sent_reduced.csv")
df_fine_tuning = pd.read_csv("finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv")
print(f"Rows that appear in both dataframes:" , df_reduced['reviewId'].isin(df_fine_tuning['reviewId']).sum())

In [ ]:
def generate_prompt_gpt3(row):
    category = row["categoryName"]
    return f"A Google Maps review about a {category}."

# def generate_prompt_gpt35turbo(row):
#     category = row["categoryName"]
#     return f"Write a Google Maps review about a {category}."

# def generate_prompt_gpt35turbo(row):
#     stars = row["stars"]
#     category = row["categoryName"]
#     keywords = row["keywords_only"]
#     keywords = keywords[1:-1].split(",")[:3]
#     keywords = [k.strip() for k in keywords]
#     keywords = ", ".join(keywords)
#     keywords = keywords.replace("'", "")
#     return f"Write a {stars} stars Google Maps review about a {category}, with the following keywords: {keywords}."

In [ ]:
df = pd.read_csv ("finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv")
print(df.shape)

df["prompt"] = df.apply(generate_prompt_gpt3, axis=1)
df["completion"] = df["text"]

print(df.shape)
print(df.columns)
print(df[['completion','prompt']].head(4999))

df.to_csv('finetuning_gpt3_sameplaces_only_english_multimod_25perplace_pc.csv', index=False)